## Prep

### import and function

In [ ]:
%matplotlib inline

# Importing libraries 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(987)

print("numpy      ", np.__version__)
print("pandas     ", pd.__version__)
print("statsmodels", sm.__version__)
print("matplotlib ", matplotlib.__version__)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)

In [ ]:
## CI function

def bootstrap_ci(treatment, formula_ps, formula_outcome, data, nb, ate, method,multilevel):
    '''
    Input: 
    treatment - column_name
    formula_ps - Regression formula for the propensity score model
    formula_outcome - Regression formula for the outcome model
    data - dataframe
    nb - number of bootstrapped samples
    ate -  ate obtained prioring to obtaining the confidence intervals
    method - "ipw" or "gcomputation"
    Output: ate - average causal effect
    
    We first obtain the propensity of the each instance to receive the treatment.
    Following this we fit the outcome model and weight the outcomes based on their propensity weights
    '''
    ate_rs = []
    for i in range(nb):  # Drawing nb bootstrapped samples, can simply start with 10 samples
        d_star = data.sample(n=data.shape[0], # Same size as input data
                             replace=True)  # Draw with replacement

        if method == "gcomputation":
            ate_ci = get_causal_effect_gcomputation(treatment,formula_outcome,d_star,multilevel)
            ate_rs.append(ate_ci)

        elif method == "ipw":
            ate_ci = get_causal_effect_ipw(treatment,d_star,formula_ps,formula_outcome,multilevel)
            ate_rs.append(ate_ci)

    ci_perc = np.percentile(ate_rs, q=[2.5, 97.5])
    return ci_perc

### data prep

#### read in data

In [ ]:
## Read preprocessed data

dat = pd.read_csv("data/processed/jhs_preprocess_0914.csv")
dat_v1 = dat[dat['visit'] == 1]
dat_v2 = dat.loc[dat['visit'] == 2, ['subjid','y']].rename(columns={'y': 'y2'})
dat_v3 = dat.loc[dat['visit'] == 3, ['subjid','y']].rename(columns={'y': 'y3'})

merged_df = pd.merge(dat_v1, dat_v2, on='subjid')
merged_df = pd.merge(merged_df, dat_v3, on='subjid')

# calculate Y
def get_Y_tot(df):
    df['y_tot'] = 0
    df.loc[
       (df['y']==1) |
       (df['y2']==1) |
       (df['y3']==1),
       'y_tot'] = 1
    return df
merged_df = get_Y_tot(merged_df)

data = merged_df

In [ ]:
## read data for dif Y

# Y_tot
data = pd.read_csv('data/processed/jhs_cox_base.csv')
data = data.rename(columns = {'event':'y_tot', 'y':'y_base'})

# Y as outcome within 5 yrs
dat_less_5y = data[data['time_12'] <= 1825]
dat_less_5y = dat_less_5y.drop(columns=['y_tot', 'y_base', 'y3', 'time_12', 'time_13'])

# Y as outcome after 5 yrs
dat_over_5y = data.drop(columns=['y_base','y_tot', 'y2', 'time_12', 'time_13'])


In [ ]:
## Select appropriate columns. 
analysis_feature = ['y3','nbSESpc2score', 'nbK3paFacilities','N_UNFAV_CT00','G_bla_rk', 
                   'PA3cat','nutrition3cat',
                   'age','gender', 'currentSmoker', 'Diabetes','sbp','hdl','totchol','alc','fmlyinc']
#dat_tot = data[analysis_feature].copy()
#dat_less_5y = dat_less_5y[analysis_feature].copy()
dat_over_5y = dat_over_5y[analysis_feature].copy()

#### combine nb effects

In [ ]:
## check sample size n
dis_sesfood = len(data.loc[(data['nbSESpc2score'] == 4.0) & (data['N_UNFAV_CT00'] == 4.0)]) 
ad_sesfood = len(data.loc[(data['nbSESpc2score'] == 1.0) & (data['N_UNFAV_CT00'] == 1.0)])
dis_sesfac = len(data.loc[(data['nbSESpc2score'] == 4.0) & (data['nbK3paFacilities'] == 1.0)]) 
ad_sesfac = len(data.loc[(data['nbSESpc2score'] == 1.0) & (data['nbK3paFacilities'] == 4.0)])

print("Sample size of nb SES&Food is {} for disadvantaged nb, {} for advantaged nb".format(dis_sesfood,ad_sesfood),"\n"
     "Sample size of nb SES&Fac is {} for disadvantaged nb, {} for advantaged nb".format(dis_sesfac,ad_sesfac))

combination_counts = data.groupby(['nbSESpc2score', 'nbK3paFacilities']).size().reset_index(name='Count')
#print(combination_counts)

In [ ]:
## combine binary exposure

dat_nb = data.copy()

dat_nb['nSESFood'] = np.where(dat_nb['nbSESpc2score'].isin([3.0, 4.0]) & dat_nb['N_UNFAV_CT00'].isin([3.0, 4.0]), 4.0, 2.5)
dat_nb.loc[dat_nb['nbSESpc2score'].isin([1.0, 2.0]) & dat_nb['N_UNFAV_CT00'].isin([1.0, 2.0]),'nSESFood'] = 1.0

dat_nb['nSESFac'] = np.where(dat_nb['nbSESpc2score'].isin([3.0, 4.0]) & dat_nb['nbK3paFacilities'].isin([1.0, 2.0]), 4.0, 2.5)
dat_nb.loc[dat_nb['nbSESpc2score'].isin([1.0, 2.0]) & dat_nb['nbK3paFacilities'].isin([3.0, 4.0]),'nSESFac'] = 1.0

## try excluding 2.5 samples
dat_nsesfood = dat_nb.loc[dat_nb['nSESFood'] != 2.5]
dat_nsesfac = dat_nb.loc[dat_nb['nSESFac'] != 2.5]

### check CVD cases by exposure 

In [ ]:
dat_outcome = dat_nb.copy()
dat_outcome.reset_index(inplace=True)

def check_outcome(exposure, data=dat_outcome):
    print(exposure)
    df = data[['index','y', exposure]]
    result_df=df.groupby([exposure,'y'])['index'].count().reset_index()
    result_df['exposure'] = exposure
    return result_df

In [ ]:
list(map(check_outcome, ['nbSESpc2score','nbK3paFacilities','N_UNFAV_CT00','G_bla_rk', 'nSESFood','nSESFac']))

## G-computation

In [ ]:
def get_causal_effect_gcomputation(treatment, formula, data,multilevel=False, outcome_dis=False):
    '''
    Input: treatment - column_name
           formula - Regression formula
           data - dataframe
           multilevel - specify if to use multilevel model for gcomputation
    Output: ate - average causal effect
    
    The main idea is to have two separate models for those with treatment=1 and treatment =0
    According to this we sample our data based on the treatment and fit two models
    
    We then predict the outcome for the entire data based on our fitted models and
    then evaluate the expected difference in the outcome which is our causal effect.
    '''
    if multilevel:
        fm_a4 = smf.mixedlm(formula, 
                        data.loc[data[treatment] == 4],groups = data.loc[data[treatment]==4]['gender']).fit()
        fm_a1 = smf.mixedlm(formula, 
                    data.loc[data[treatment] == 1], groups = data.loc[data[treatment]==1]['gender']).fit()

    else:
        f = sm.families.family.Binomial()
        fm_a4 = smf.glm(formula, 
                        data.loc[data[treatment] == 4], family=f).fit()
        fm_a1 = smf.glm(formula, 
                    data.loc[data[treatment] == 1], family=f).fit()

    y_a4 = fm_a4.predict(data)
    y_a1 = fm_a1.predict(data)
    ate = np.mean(y_a4 - y_a1)
    
    if outcome_dis:
        plt_data = {'Group': np.concatenate([['Exp_4']*len(y_a4), ['Exp_1']*len(y_a1)]),
                'Value': np.concatenate([y_a4, y_a1]) }
        plt_df = pd.DataFrame(plt_data)
        sns.kdeplot(data=plt_df, x='Value', hue='Group', fill=True, common_norm=False)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.title(f"Outcome distribution for exposure {treatment}")
        return ate,plt
    
    else: 
        return ate

In [ ]:
treatment = "nbSESpc2score"
formula_g = "y_tot ~ C(nbK3paFacilities)+C(N_UNFAV_CT00)+C(G_bla_rk)\
                 +C(PA3cat)+C(nutrition3cat)+\
                age + sbp + hdl + totchol + C(currentSmoker) + C(Diabetes)+C(alc)+C(fmlyinc)"

## formula_ps only for the ci function, not used in gcomputation
formula_ps = "nbK3paFacilities ~ C(N_UNFAV_CT00)+C(nbSESpc2score)+C(G_bla_rk)+\
                C(sportIndex)+C(hyIndex)+C(activeIndex)+ C(darkgrnVeg)+C(eggs)+C(fish)+\
                age + sbp + hdl + totchol + C(currentSmoker) + C(Diabetes)"

### single neighborhood exposure

In [ ]:
print("Obtaining causal effect using gcomputation multilevel")

effect_g = get_causal_effect_gcomputation(treatment,formula_g,data,multilevel=True, outcome_dis=False)
ci_g = bootstrap_ci(treatment,formula_ps,formula_g,data,100,effect_g,"gcomputation",multilevel=True)
print("ATE from Gcomputation is {} with confidence interval {}".format(np.round(effect_g,10),ci_g))


In [ ]:
print("Obtaining outcome distribution")

ate_g, plt_g = get_causal_effect_gcomputation(treatment,formula_g,data,multilevel=True, outcome_dis=True)
plt_g


### combined neighborhood effects

In [ ]:
# dat_nb (n=3568), dat_nsesfood, dat_nsesfac

print("Obtaining causal effect of combined nb features using gcomputation multilevel")
effect_g= get_causal_effect_gcomputation(treatment,formula_g,dat_nsesfood,multilevel=True, outcome_dis=False)
ci_g = bootstrap_ci(treatment,formula_ps,formula_g,dat_nsesfood,100,effect_g,"gcomputation",multilevel=True)
print("ATE from Gcomputation is {} with confidence interval {}".format(np.round(effect_g,10),ci_g))


In [ ]:
print("Obtaining outcome distribution for combined effects")

ate_g, plt_g = get_causal_effect_gcomputation(treatment,formula_g,dat_nsesfac,multilevel=True, outcome_dis=True)
plt_g

### individual exposure

In [ ]:
# change value to 1/4 to use the function
data_ind = data.copy()

data_ind.loc[data_ind['PA3cat']==0,'PA3cat'] = 1
data_ind.loc[data_ind['PA3cat']==2,'PA3cat'] = 4

dat_phy = data_ind.loc[data_ind['PA3cat'] != 2.0]

data_ind.loc[data_ind['nutrition3cat']==1,'nutrition3cat'] = 4
data_ind.loc[data_ind['nutrition3cat']==2,'nutrition3cat'] = 4
data_ind.loc[data_ind['nutrition3cat']==0,'nutrition3cat'] = 1

In [ ]:
print("Obtaining causal effect using gcomputation multilevel")

effect_g = get_causal_effect_gcomputation(treatment,formula_g,data_ind,multilevel=True, outcome_dis=False)
effect_g
ci_g = bootstrap_ci(treatment,formula_ps,formula_g,data_ind,100,effect_g,"gcomputation",multilevel=True)
print("ATE from Gcomputation is {} with confidence interval {}".format(np.round(effect_g,10),ci_g))


## IPW

In [ ]:
def get_causal_effect_ipw(treatment,data,formula_ps,formula_outcome,multilevel=False, outcome_dis=False):
    '''
    Input: 
    treatment - column_name
    data - dataframe
    formula_ps - Regression formula for the propensity score model
    formula_outcome - Regression formula for the outcome model
    multilevel - specify if to use a multilevel model for estimating propensity scores
    Output: ate - average causal effect
    
    We first obtain the propensity of the each instance to receive the treatment.
    Following this we fit the outcome model and weight the outcomes based on their propensity weights
    '''
    data['gender'] = pd.to_numeric(data['gender'])
    f = sm.families.family.Binomial()
    if multilevel:
        ps_model = smf.mixedlm(formula_ps, 
                        data,groups = data['gender']).fit()
    else:
        ps_model = smf.glm(formula_ps,data,family=f).fit()
    
    ps_scores = ps_model.predict(data)
        
    outcome_model = smf.glm(formula_outcome,data,family=f).fit()
    predicted_outcomes = outcome_model.predict(data)
    
    treatment_4 = np.where(data[treatment] == 4, 1, 0)
    treatment_1 = np.where(data[treatment] == 1, 1, 0)
    
    potential_outcome4 = treatment_4*predicted_outcomes/ps_scores / sum(data[treatment]/ps_scores)
    potential_outcome1 = treatment_1*predicted_outcomes/(1-ps_scores) / sum((1-data[treatment])/(1-ps_scores))
    ate = potential_outcome4.sum() - potential_outcome1.sum()
    
    if outcome_dis:
        plt_data = {'Group': np.concatenate([['Exp_4']*len(potential_outcome4), ['Exp_1']*len(potential_outcome1)]),
                    'Value': np.concatenate([potential_outcome4, potential_outcome1]) }
        plt_df = pd.DataFrame(plt_data)
        sns.kdeplot(data=plt_df, x='Value', hue='Group', fill=True, common_norm=False)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.title(f"Outcome distribution for exposure {treatment}")
        return ate,plt
    
    else: 
        return ate

In [ ]:
treatment = 'nbSESpc2score'
outcome = 'y3'
formula_outcome = "y3 ~ 1+ C(nbSESpc2score)"
formula_ps = "nbSESpc2score ~ C(nbK3paFacilities)+C(N_UNFAV_CT00)+C(G_bla_rk)+\
                C(PA3cat)+C(nutrition3cat)+\
                age + sbp + hdl + totchol + C(currentSmoker) + C(Diabetes)+C(alc)+C(fmlyinc)"


### single exposure

In [ ]:
print("Obtaining causal effect using IPW multilevel")
effect_ipw= get_causal_effect_ipw(treatment,dat_over_5y,formula_ps,formula_outcome,multilevel=True, outcome_dis=False)
ci_ipw = bootstrap_ci(treatment,formula_ps,formula_outcome,dat_over_5y,100,effect_ipw,"ipw",multilevel=True)
print("ATE from multilevel IPW is {} with confidence interval {}".format(np.round(effect_ipw,10),ci_ipw))

In [ ]:
print("Obtaining outcome distribution for combined effects")

ate_ipw, plt_ipw = get_causal_effect_ipw(treatment,data,formula_ps,formula_outcome,multilevel=True, outcome_dis=True)
plt_ipw

### combined neighborhood effects

In [ ]:
# dat_nb (n=3568), dat_nsesfood, dat_nsesfac

print("Obtaining causal effect using IPW multilevel")
effect_ipw= get_causal_effect_ipw(treatment,dat_nsesfac,formula_ps,formula_outcome,multilevel=True)
ci_ipw = bootstrap_ci(treatment,formula_ps,formula_outcome,dat_nsesfac,100,effect_ipw,"ipw",multilevel=True)
print("ATE from multilevel IPW is {} with confidence interval {}".format(np.round(effect_ipw,10),ci_ipw))

In [ ]:
print("Obtaining outcome distribution for combined effects")

ate_ipw, plt_ipw = get_causal_effect_ipw(treatment,data,formula_ps,formula_outcome,multilevel=True, outcome_dis=True)
plt_ipw

### individual exposure

In [ ]:
## data_ind, dat_phy
print("Obtaining causal effect using IPW multilevel")
effect_ipw= get_causal_effect_ipw(treatment,data_ind,formula_ps,formula_outcome,multilevel=True)
ci_ipw = bootstrap_ci(treatment,formula_ps,formula_outcome,data_ind,100,effect_ipw,"ipw",multilevel=True)
print("ATE from multilevel IPW is {} with confidence interval {}".format(np.round(effect_ipw,10),ci_ipw))

## others

### extreme values for ipw rs

In [ ]:
# data = data.sample(n=data.shape[0], replace=True)

data['gender'] = pd.to_numeric(data['gender'])
f = sm.families.family.Binomial()
formula_ps = "G_bla_rk ~ C(N_UNFAV_CT00)+C(nbK3paFacilities)+C(nbSESpc2score)+\
                C(sportIndex)+C(hyIndex)+C(activeIndex)+ C(darkgrnVeg)+C(eggs)+C(fish)+\
                age + sbp + hdl + totchol + C(currentSmoker) + C(Diabetes)"
ps_model = smf.mixedlm(formula_ps, data,groups = data['gender']).fit()

ps_scores = ps_model.predict(data)

formula_outcome = "y ~ 1+ C(G_bla_rk)"
outcome_model = smf.glm(formula_outcome,data,family=f).fit()
predicted_outcomes = outcome_model.predict(data)

treatment = 'G_bla_rk' 
treatment_4 = np.where(data[treatment] == 4, 1, 0)
treatment_1 = np.where(data[treatment] == 1, 1, 0)
    
potential_outcome4 = treatment_4*predicted_outcomes/ps_scores / sum(data[treatment]/ps_scores)
potential_outcome1 = treatment_1*predicted_outcomes/(1-ps_scores) / sum((1-data[treatment])/(1-ps_scores))
ate = potential_outcome4.sum() - potential_outcome1.sum()
ate

In [ ]:
def check_extreme(data_array):

    # Check for extreme values based on threshold
    z_scores = (data_array - np.mean(data_array)) / np.std(data_array)
    threshold = 3
    extreme_value_index = np.where(abs(z_scores) > threshold)[0]
    
    return extreme_value_index

ind = check_extreme(potential_outcome4)
print(ind)

In [ ]:
## new ipw function excluding extreme value

def ipw_drop_extreme(treatment,data,formula_ps,formula_outcome,multilevel=False, outcome_dis=False):
    
    data['gender'] = pd.to_numeric(data['gender'])
    f = sm.families.family.Binomial()
    if multilevel:
        ps_model = smf.mixedlm(formula_ps, 
                        data,groups = data['gender']).fit()
    else:
        ps_model = smf.glm(formula_ps,data,family=f).fit()
    
    ps_scores = ps_model.predict(data)
        
    outcome_model = smf.glm(formula_outcome,data,family=f).fit()
    predicted_outcomes = outcome_model.predict(data)
    
    treatment_4 = np.where(data[treatment] == 4, 1, 0)
    treatment_1 = np.where(data[treatment] == 1, 1, 0)
    
    potential_outcome4 = treatment_4*predicted_outcomes/ps_scores / sum(data[treatment]/ps_scores)
    potential_outcome1 = treatment_1*predicted_outcomes/(1-ps_scores) / sum((1-data[treatment])/(1-ps_scores))
    
    # drop extreme predicted values
    ind_4 = check_extreme(potential_outcome4)
    potential_outcome4_drop = np.delete(np.array(potential_outcome4), ind_4)
    ind_1 = check_extreme(potential_outcome1)
    potential_outcome1_drop = np.delete(np.array(potential_outcome1), ind_1)
    ate = potential_outcome4_drop.sum() - potential_outcome1_drop.sum()
    
    if outcome_dis:
        plt_data = {'Group': np.concatenate([['Exp_4']*len(potential_outcome4_drop), ['Exp_1']*len(potential_outcome1_drop)]),
                    'Value': np.concatenate([potential_outcome4_drop, potential_outcome1_drop]) }
        plt_df = pd.DataFrame(plt_data)
        sns.kdeplot(data=plt_df, x='Value', hue='Group', fill=True, common_norm=False)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.title(f"Outcome distribution for exposure {treatment}")
        return ate,plt
    
    else: 
        return ate

In [ ]:
## CI function with extreme value drop

def bootstrap_ci(treatment, formula_ps, formula_outcome, data, nb, ate, method,multilevel):

    ate_rs = []
    for i in range(nb):  # Drawing nb bootstrapped samples, can simply start with 10 samples
        d_star = data.sample(n=data.shape[0], # Same size as input data
                             replace=True)  # Draw with replacement

        if method == "gcomputation":
            ate_ci = get_causal_effect_gcomputation(treatment,formula_outcome,d_star,multilevel)
            ate_rs.append(ate_ci)

        elif method == "ipw":
            ate_ci = ipw_drop_extreme(treatment,d_star,formula_ps,formula_outcome,multilevel)
            ate_rs.append(ate_ci)

    ci_perc = np.percentile(ate_rs, q=[2.5, 97.5])
    return ci_perc

In [ ]:
treatment = 'G_bla_rk'
outcome = 'y'
formula_outcome = "y ~ 1+ C(G_bla_rk)"
formula_ps = "G_bla_rk ~ C(N_UNFAV_CT00)+C(nbK3paFacilities)+C(nbSESpc2score)+\
                C(sportIndex)+C(hyIndex)+C(activeIndex)+ C(darkgrnVeg)+C(eggs)+C(fish)+\
                age + sbp + hdl + totchol + C(currentSmoker) + C(Diabetes)"

In [ ]:
print("Obtaining causal effect using IPW multilevel")
effect_ipw= ipw_drop_extreme(treatment,data,formula_ps,formula_outcome,multilevel=True, outcome_dis=False)
ci_ipw = bootstrap_ci(treatment,formula_ps,formula_outcome,data,100,effect_ipw,"ipw",multilevel=True)
print("ATE from multilevel IPW is {} with confidence interval {}".format(np.round(effect_ipw,10),ci_ipw))

In [ ]:
print("Obtaining outcome distribution for combined effects")

ate_ipw, plt_ipw = ipw_drop_extreme(treatment,data,formula_ps,formula_outcome,multilevel=True, outcome_dis=True)
plt_ipw